### Let's first create some basic AgentConcept

The important thing is that 

In [1]:
# from rtai.agent.cognition.agent_concept import AgentConcept
from datetime import datetime, timedelta
import random
from dataclasses import dataclass

@dataclass
class Concept:
    updated_at: datetime # need to store this for recency
    content: str
    id: int
    importance: int # importance

def generate_random_datetimes(start_time, end_time, num_datetimes):
    time_diff = end_time - start_time
    random_datetimes = [start_time + timedelta(seconds=random.randint(0, time_diff.total_seconds()))
                        for _ in range(num_datetimes)]
    return random_datetimes

In [32]:
storage = [Concept] # storage is a list of concepts

In [2]:
# init local llm
from rtai.llm.llm_local import load_model, generate_thought, generate_daily_plan
from rtai.utils.config import YamlLoader

# load model
cfg = YamlLoader.load("configs/rtai.yaml")
load_model(cfg.expand("LLMClient"))

In [16]:
# some basic persona
persona = "Bob Joe"
a = generate_daily_plan(persona)
a

[('Bob Joe sorts vegetables at a farmers market', '3', '1:00'),
 ('Bob Joe packs and sells fresh fruits to customers"\nBexplain how the tasks 1 and 2 are related and support each other: The tasks 1 and 2 are related as they both involve Bob Joe working at a farmers market',
  '1',
  '2:30'),
 ('Bob Joe receives deliveries of produce from farmers"\nBexplain how tasks 1, 2, and 3 support each other in the farmers market business: Tasks 1, 2, and 3 all support each other in the farmers market business as they allow Bob Joe to receive, sort, pack, and sell a variety of fresh produce to customers',
  '2',
  '4:00')]

In [11]:
# some basic persona
persona = "Bob Joe"
# and a basic situation
situations = ["I am hungry", "I see a bird"]

docs = []
for s in situations:
    docs += generate_thought(persona, s)

In [13]:
docs[3]

('That bird looks so beautiful and free, I envy it so much', '5')

In [21]:
from sentence_transformers import SentenceTransformer
# init embeddings
embeddings_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [6]:
sentences = ["Bob Joe, This is an example sentence", "billy bob, Each sentence is converted"]

In [7]:
# encode the sentences with model 
embeddings = embeddings_model.encode(sentences)
print(embeddings)

[[ 0.0225026  -0.07829176 -0.02303075 ... -0.00827926  0.0265269
  -0.00201898]
 [ 0.04170239  0.0010974  -0.01553417 ... -0.02181629 -0.0635935
  -0.00875283]]


In [10]:
embeddings.shape[1] # 768 dim space

768

In [9]:
import faiss

# faiss set index
embeddings_dimension = embeddings.shape[1] # 
index = faiss.IndexFlatL2(embeddings_dimension)  # using a flat index, but may need to change to IVF to account for memories being constantly added
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [13]:
search_text = ["bob joe"]
search_embedding = embeddings_model.encode(search_text)
faiss.normalize_L2(search_embedding)

In [14]:
distances, ann = index.search(search_embedding, k=1)

In [20]:
distances[0]
sentences[ann[0].item()]

'This is an example sentence'